### Imports

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score
import matplotlib.pyplot as plt
import seaborn as sns #???

### Model Definition

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=dilation, dilation=dilation)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.resample = nn.Conv1d(in_channels, out_channels, 1) if in_channels!= out_channels else nn.Identity()

    def forward(self, x):
        residual = self.resample(x)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.dropout(x)
        return x + residual
    

class TCNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, time_dim, out_channels, num_classes, seq_len):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.time_fc = nn.Linear(3, embed_dim)
        
        self.blocks = nn.Sequential(
            ResidualBlock(embed_dim, out_channels, kernel_size=3, dilation=1),
            ResidualBlock(out_channels, out_channels, kernel_size=3, dilation=2),
            ResidualBlock(out_channels, out_channels, kernel_size=3, dilation=4)
        )
        
        self.pool = nn.AdaptiveAvgPool1d(1) #Global average pooling
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(out_channels, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )
        
    def forward(self, seq_input, time_input):
        x_seq = self.embedding(seq_input)
        
        B, L, _ = time_input.shape
        x_time = self.time_fc(time_input.view(B * L, -1))
        x_time = x_time.view(B, L, -1)
        
        x = x_seq + x_time
        x = x.permute(0, 2, 1)
        x = self.blocks(x)
        x = self.pool(x)
        return self.classifier(x)

### Data Loading

In [ ]:
data_dir = "./dataTCN/"

X_seq = np.load(data_dir + "X_seq.npy")       # [B, L] endpoints
X_time = np.load(data_dir + "X_time.npy")     # [B, L, 3] times
y = np.load(data_dir + "y.npy")               # [B] user classes

print("X_seq:", X_seq.shape)
print("X_time:", X_time.shape)
print("y:", y.shape)

### Train/Test Split

In [ ]:
X_seq_train, X_seq_test, X_time_train, X_time_test, y_train, y_test = train_test_split(
    X_seq, X_time, y, test_size=0.2, random_state=169783, stratify=y
)

### Datasets and DataLoaders

In [ ]:
X_seq_train = torch.tensor(X_seq_train, dtype=torch.long)
X_seq_test = torch.tensor(X_seq_test, dtype=torch.long)

X_time_train = torch.tensor(X_time_train, dtype=torch.float32)
X_time_test = torch.tensor(X_time_test, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

batch_size = 64

# Datasets
train_ds = TensorDataset(X_seq_train, X_time_train, y_train)
test_ds = TensorDataset(X_seq_test, X_time_test, y_test)

# DataLoaders
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=batch_size)

### Model Initialization

In [ ]:
vocab_size = int(X_seq.max() + 1)
embed_dim = 64
out_channels = 64
num_classes = int(y.max() + 1)
seq_len = X_seq.shape[1]

model = TCNClassifier(vocab_size, embed_dim, 3, out_channels, num_classes, seq_len)

### Training Loop

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

n_epochs = 10
train_losses = []
train_accuracies = []

for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for seq_batch, time_batch, y_batch in train_loader:
        seq_batch = seq_batch.to(device)
        time_batch = time_batch.to(device)
        y_batch = y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(seq_batch, time_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)
        
    acc = correct / total
    train_losses.append(total_loss / len(train_loader))
    train_accuracies.append(acc)
    
    print(f"\nEpoch {epoch + 1}/{n_epochs} - Loss: {train_losses[-1]:.4f} - Accuracy: {acc:.4f}")

### Evaluation

In [ ]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for seq_batch, time_batch, y_batch in test_loader:
        seq_batch = seq_batch.to(device)
        time_batch = time_batch.to(device)
        outputs = model(seq_batch, time_batch)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(y_batch.numpy())

print("Classification report:")
print(classification_report(all_labels, all_preds))

### Saving the Model

In [ ]:
model_path = "./models/tcn_model.pt"
torch.save(model.state_dict(), model_path)
print(f"Model saved in {model_path}")

### Visualization

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title("TCN - Confusion Matrix")
plt.show()

In [ ]:
f1_per_class = f1_score(all_labels, all_preds, average=None)
labels_unique = np.unique(y)

plt.bar(range(len(f1_per_class)), f1_per_class, tick_label=labels_unique)
plt.title("TCN - F1-score per Class")
plt.xlabel("Class")
plt.ylabel("F1-score")
plt.ylim(0, 1)
plt.grid(True)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

axs[0].plot(range(1, n_epochs + 1), train_losses, marker='o')
axs[0].set_title("Training Loss per Epoch")
axs[0].set_xlabel("Epoch")
axs[0].set_ylabel("Loss")
axs[0].grid(True)

axs[1].plot(range(1, n_epochs + 1), train_accuracies, marker='o', color='green')
axs[1].set_title("Training Accuracy per Epoch")
axs[1].set_xlabel("Epoch")
axs[1].set_ylabel("Accuracy")
axs[1].grid(True)

plt.tight_layout()
plt.show()